In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
passengers1 = pd.read_csv('data/passenger2016_2017.csv')
passengers2 = pd.read_csv('data/passenger_2018.csv')
fuel_con = pd.read_csv('data/Fuel_consumption.csv')

In [ ]:
#passengers by air carrier
passengers_bycarrier = passengers[passengers['year']>2017][['carrier','passengers']].groupby('carrier').sum().sort_values('passengers',ascending=False)

In [ ]:
#total fuel consumption per air carrier
fuel_con_bycarrier = fuel_con[fuel_con['year']>2017][['carrier','total_gallons']].groupby('carrier').sum().sort_values('total_gallons',ascending=False)

In [ ]:
df = pd.read_csv('data/model_df_version_1.csv')

In [ ]:
df['month'] = pd.DatetimeIndex(df['fl_date']).month
df['weekday'] = pd.DatetimeIndex(df['fl_date']).weekday
df['year'] = pd.DatetimeIndex(df['fl_date']).year

In [ ]:
flights_by_carriermonth = df[['mkt_unique_carrier','month','year','distance','dep_delay']].groupby(['mkt_unique_carrier','month','year']).sum()

In [ ]:
average_by_carrier = flights_by_carriermonth[['distance','dep_delay']].groupby('mkt_unique_carrier').mean()

In [ ]:
qtendf = average_by_carrier.merge(fuel_con_bycarrier,how = 'left', left_index = True, right_index=True)
qtendf = qtendf.merge(passengers_bycarrier,how = 'left', left_index = True, right_index=True)

In [ ]:
qtendf.head()

In [ ]:
median_gallon = qtendf[qtendf['total_gallons']>0]['total_gallons'].median()

In [ ]:
qtendf['passengers'] = qtendf['passengers']/24

In [ ]:
qtendf['total_gallons'] = qtendf['total_gallons'].where(qtendf['total_gallons']>0,avg_gallon)
qtendf['monthly_distance_per_passenger'] = (qtendf['passengers']/qtendf['distance'])
qtendf['avgfuel_percustomer_perdistance'] = qtendf['total_gallons']/qtendf['monthly_distance_per_passenger']
qtendf['delay_ratio'] = qtendf['dep_delay']/qtendf['avgfuel_percustomer_perdistance']*10000

In [ ]:
qtendf[qtendf['total_gallons']>0][['dep_delay','delay_ratio','avgfuel_percustomer_perdistance']].sort_values(by='dep_delay',ascending=False)

In [ ]:
plt.scatter(qtendf['dep_delay'], qtendf['avgfuel_percustomer_perdistance'])

In [ ]:
qtendf = qtendf.rename(columns={'distance':'avg_distance_per_month_by_carrier','dep_delay':'avg_dep_delay_by_carrier','passengers':'passengers_by_carrier_per_month'})

In [ ]:
plt.scatter(qtendf['avg_dep_delay_by_carrier'],qtendf['passengers_by_carrier_per_month'])

In [ ]:
plt.scatter(qtendf['avg_dep_delay_by_carrier'],qtendf['total_gallons'])

In [ ]:
qtendf = qtendf.drop(columns='delay_ratio')